# Обработчик файлов

In [241]:
pip install yadisk


In [242]:
pip install openpyxl

In [243]:
# Импортируем библиотеки
import yadisk
import os
import pandas as pd
import re
from tqdm import tqdm
from openpyxl import load_workbook
from google.colab import files
import configparser

In [ ]:
# Загружаем файл из сессионного хранилища
uploaded = files.upload()

# Создаем объект конфигурации
config = configparser.ConfigParser()

config.read('credentials.ini')

app_id = config['Credentials']['app_id']
secret_id = config['Credentials']['secret_id']
ya_token = config['Credentials']['ya_token']

In [245]:
# Подключаемся к Яндекс Диску
y = yadisk.YaDisk(app_id, secret_id, ya_token)


In [246]:
# Создаем список файлов для загрузки через списочное выражение, чтоб попали файлы с расширением xlsx
list_of_files = []

for el in tqdm(list(y.listdir('file_processor'))):
  if el['path'].endswith('.xlsx'):
    list_of_files.append(el['path'])



100%|██████████| 148/148 [00:00<00:00, 145308.28it/s]


In [247]:
# Создаем папку files для загрузки
load_path = '/content/files/'


In [248]:
# Загружаем файлы с яндекс диска
for file in tqdm(list_of_files):
 y.download(file.split(':')[1], file.split('/')[-1])


100%|██████████| 147/147 [04:19<00:00,  1.77s/it]


In [249]:
# Закачиваем файлы на диск
list_of_files[:3]

['disk:/file_processor/2021_Таблица_005_Злокачественные_новообразования_в_РФ_(заболеваемость_и_смертность).xlsx',
 'disk:/file_processor/2021_Таблица_006_Злокачественные_новообразования_в_РФ_(заболеваемость_и_смертность).xlsx',
 'disk:/file_processor/2021_Таблица_007_Злокачественные_новообразования_в_РФ_(заболеваемость_и_смертность).xlsx']

In [250]:
list_of_files[0].split('/')

['disk:',
 'file_processor',
 '2021_Таблица_005_Злокачественные_новообразования_в_РФ_(заболеваемость_и_смертность).xlsx']

In [251]:
#  Создадим 4 списка с номерами файлов, которые будем объединять, с помощью zfill() добавим нули в начало числа.
#  3 датафрейм будет сфоомирован из files_list_free + files_list_three_no_gender. Так как таблицы из files_list_three_no_gender не содержат сведения с разбивкой по полу.
#  4 датафрейм будет сфоомирован из files_list_four + files_list_four_no_gender. Так как таблицы из files_list_four_no_gender не содержат сведения с разбивкой по полу.

files_list_one = [58] + list(range(61, 86))
files_list_two = [24] + list(range(27, 52))
files_list_three = list(range(12, 36)) + list(range(45,60))
files_list_three_no_gender = list(range(36,45))
files_list_four = list(range(69, 87)) + list(range(93,104))
files_list_four_no_gender = list(range(87,93))

lists = [
     files_list_one,
     files_list_two,
     files_list_three,
     files_list_three_no_gender,
     files_list_four,
     files_list_four_no_gender
]

for file_list in lists:
    for i in range(len(file_list)):
        file_list[i] = str(file_list[i]).zfill(3)



In [252]:
convert_one = []
convert_two = []
convert_three = []
convert_three_dop = []
convert_four = []
convert_four_dop = []

In [253]:
for file in os.listdir():
    file_parts = file.split('_')
    if len(file_parts) >= 3:
        if str(file_parts[2]) in files_list_one and 'Сост' in file:
            convert_one.append(file)
        else:
            pass
    else:
        pass



In [254]:
for file in os.listdir():
    file_parts = file.split('_')
    if len(file_parts) >= 3:
        if str(file_parts[2]) in files_list_two and 'Сост' in file:
            convert_two.append(file)
        else:
            pass
    else:
        pass


In [255]:
for file in os.listdir():
    file_parts = file.split('_')
    if len(file_parts) >= 3:
        if str(file_parts[2]) in files_list_three and 'Зло' in file:
            convert_three.append(file)
        else:
            pass
    else:
        pass




In [256]:
for file in os.listdir():
    file_parts = file.split('_')
    if len(file_parts) >= 3:
        if str(file_parts[2]) in files_list_three_no_gender and 'Зло' in file:
            convert_three_dop.append(file)
        else:
            pass
    else:
        pass



In [257]:
for file in os.listdir():
    file_parts = file.split('_')
    if len(file_parts) >= 3:
        if str(file_parts[2]) in files_list_four and 'Зло' in file:
            convert_four.append(file)
    else:
        pass
else:
    pass


In [258]:
for file in os.listdir():
    file_parts = file.split('_')
    if len(file_parts) >= 3:
        if str(file_parts[2]) in files_list_four_no_gender and 'Зло' in file:
            convert_four_dop.append(file)
    else:
        pass
else:
    pass


В переменной final_one сохраним таблицы из списка convert_one

In [259]:
final_one = []

In [260]:
for el in tqdm(convert_one):
    vars = load_workbook(el).active['A1'].value.split('  ')
    ind, loc = ' '.join(vars[0].split('\n')[0:2]).capitalize(), vars[0].split('\n')[2]
    table = vars[-1].strip()
    df = pd.read_excel(el, header=[2])
    year = re.search(r'\b\d{4}\b', vars[0]).group()
    df.columns = ['Регион',
                  'Число ЗНО, выявленных в отчетном году, радикальное лечение которых закончено в отчетном году, чел.',
                  'Число ЗНО, выявленных в отчетном году, радикальное лечение которых закончено в отчетном году, % от впервые выявленных',
                  'Число ЗНО, выявленных в отчетном году, радикальное лечение которых будет продолжено в отчетом году, чел.',
                  'Число ЗНО, выявленных в отчетном году, радикальное лечение которых будет продолжено в отчетном году, % от впервые выявленных',
                  'В том числе с использованием методов только хирургического, %',
                  'В том числе с использованием методов только лучевого, %',
                  'В том числе с использованием методов только лекарственного, %',
                  'В том числе с использованием методов комбинированного или комплексного (кроме химиолучевого), %',
                  'В том числе с использованием методов химиолучевого, %']
    df['Индикатор'] = ind
    df['Локализация'] = loc
    df['Таблица'] = table
    df['Тип данных'] = 'СОП зарегистрировано'
    df['Год'] = year
    final_one.append(df)
else:
    print("Некорректный формат значения ячейки А1 в файле:", file)

100%|██████████| 26/26 [00:01<00:00, 13.48it/s]

Некорректный формат значения ячейки А1 в файле: sample_data


In [261]:
#  в final_one_df объединим все таблицы из спика final_one
final_one_df = pd.concat(final_one)


In [262]:
#  создадим словарь с Федеральными округами России и входящими в них субъектами
fed_districts = {'ЦЕНТРАЛЬНЫЙ ФО' :
               ['Белгородская область', 'Брянская область', 'Владимирская область', 'Воронежская область', 'Ивановская область',\
                'Калужская область', 'Костромская область', 'Курская область', 'Липецкая область', 'Московская область', 'Орловская область', \
                'Рязанская область', 'Смоленская область', 'Тамбовская область', 'Тверская область', 'Тульская область', 'Ярославская область', 'г. Москва'],
               'СЕВЕРО-ЗАПАДНЫЙ ФО' :
               ['Республика Карелия', 'Республика Коми', 'Архангельская обл.(б/а.о)', 'Вологодская область', 'Калининградская область', 'Ленинградская область', \
                'Мурманская область', 'Новгородская область', 'Псковская область', 'г. Санкт-Петербург', \
                'Ненецкий а.о.'],
               'ЮЖНЫЙ ФО' :
               ['Краснодарский край', 'Астраханская область', 'Волгоградская область', 'Ростовская область', 'Республика Адыгея', 'Республика Калмыкия',\
                'Республика Крым', 'Город Cевастополь'],
               'СЕВЕРО-КАВКАЗСКИЙ ФО' :
               ['Ставропольский край', 'Республика Ингушетия', 'Республика Дагестан', 'Респ. Кабардино-Балкария', 'Респ. Северная Осетия',\
                'Респ. Карачаево-Черкесия', 'Республика Чечня'],
               'ПРИВОЛЖСКИЙ ФО' :
               ['Нижегородская область', 'Кировская область', 'Самарская область', 'Оренбургская область', 'Пензенская область',\
                'Пермский край', 'Саратовская область', 'Ульяновская область', 'Республика Башкортостан', 'Республика Марий Эл',\
                'Республика Мордовия', 'Республика Татарстан', 'Республика Удмуртия', 'Республика Чувашия'],
               'УРАЛЬСКИЙ ФО' :
               ['Ханты-Мансийский а.о.', 'Ямало-Ненецкий а.о.', 'Курганская область', 'Свердловская область', 'Тюменская обл.(б/а.о)',\
                'Челябинская область'],
               'СИБИРСКИЙ ФО' :
               ['Алтайский край', 'Красноярский край', 'Иркутская область', 'Кемеровская область', 'Новосибирская область', 'Омская область',\
                'Томская область', 'Республика Алтай', 'Республика Тыва', 'Республика Хакасия'],
               'ДАЛЬНЕВОСТОЧНЫЙ ФО' :
               ['Приморский край', 'Хабаровский край', 'Амурская область', 'Камчатский край', 'Магаданская область', 'Сахалинская область', 'Забайкальский край',\
                'Чукотский авт.округ', 'Республика Бурятия', 'Республика Саха (Якутия)', 'Еврейская авт. обл.']}




In [263]:
#  Удаление пробелов в начале и конце строк в столбце регион
final_one_df.loc[:, 'Регион'] = final_one_df['Регион'].str.strip()

In [264]:
# Удаление строки с названием "Россия"
final_one_df = final_one_df[final_one_df['Регион'] != 'РОССИЯ']

In [265]:
# Извлекаем все строки, содержащие 'ФО'
bad_df = final_one_df[final_one_df['Регион'].str.upper().str.contains('ФО', regex=True)]

# Удаляем строки из final_one_df, которые есть в bad_df
final_one_df = final_one_df[~final_one_df['Регион'].isin(bad_df['Регион'])]


In [266]:
# Создание нового столбца 'ФО' на основе существующего столбца 'Регион' с использованием лямбда-функции
final_one_df['ФО'] = final_one_df['Регион'].apply(lambda x: next((fd for fd, subs in fed_districts.items() if x in subs), None))

In [267]:
final_two = []

for file in convert_two:
    workbook = openpyxl.load_workbook(file)
    first_sheet = workbook.worksheets[0]
    cell_value = first_sheet['A1'].value.split('  ')

    if len(cell_value) >= 4:
        ind = first_sheet['A1'].value.split('\n')[0].split('В 2021 Г.')[0]
        loc = first_sheet['A1'].value.split('\n')[-1].split('Таблица')[0].strip()
        table = first_sheet['A1'].value.split('  ')[-1].split('\n')[0].strip()
        year =  first_sheet['A1'].value.split('Г.')[0].split(' ')[-2]

        df = pd.read_excel(file, header=[2])
        df.columns = [
            'Регион',
            'Взято на учет больных с впервые в жизни уст. диагнозом ЗНО',
            'в т.ч. вы- явлены активно, %',
            'Находились на учете на конец года, абсолютное число',
            'Находились на учете на конец года, на 100 тыс. населения',
            'Из них 5 лет и более, абсолютное число',
            'Из них 5 лет и более, %  от сост. на учете',
            'Индекс накопления контингентов',
            'Летальность, %'
        ]

        df['Индикатор'] = ind
        df['Локализация'] = loc
        df['Таблица'] = table
        df['Тип данных'] = 'СОП зарегистрировано'
        df['Год'] = year

        final_two.append(df)
    else:
        print("Некорректный формат значения ячейки А1 в файле:", file)



Некорректный формат значения ячейки А1 в файле: 2021_Таблица_032_Состояние_онко_помощи_в_РФ.xlsx


In [268]:
final_two_df = pd.concat(final_two)


In [269]:
#  Удаление пробелов в начале и конце строк в столбце регион
final_two_df.loc[:, 'Регион'] = final_two_df['Регион'].str.strip()
# Удаление строки с названием "Россия"
final_two_df = final_two_df[final_two_df['Регион'] != 'РОССИЯ']
# Извлекаем все строки, содержащие 'ФО'
bad_df = final_two_df[final_two_df['Регион'].str.upper().str.contains('ФО', regex=True)]
# Удаляем строки из final_two_df, которые есть в bad_df
final_two_df = final_two_df[~final_two_df['Регион'].isin(bad_df['Регион'])]
# Создание нового столбца 'ФО' на основе существующего столбца 'Регион' с использованием лямбда-функции
final_two_df['ФО'] = final_two_df['Регион'].apply(lambda x: next((fd for fd, subs in fed_districts.items() if x in subs), None))


In [270]:
#  Некорректно обработалась Таблица 32
#  Придется добавлять ее
final_two_df.loc[final_two_df['Таблица'] == '', 'Таблица'] = 'Таблица 32'


In [271]:
#  обработаем в каждом файле из второго списка вторые листы:
final_two_dop = []

for file in convert_two:
    workbook = openpyxl.load_workbook(file)
    first_sheet = workbook.worksheets[-1]
    cell_value = first_sheet['A1'].value.split('  ')

    if len(cell_value) >= 4:
        ind = first_sheet['A1'].value.split(',')[0]
        loc = first_sheet['A1'].value.split('\n')[-1].split('Таблица')[0].strip()
        table = cell_value[-1].strip()
        year =  first_sheet['A1'].value.split('Г.')[0].split(' ')[-2]

        df = pd.read_excel(file, header=[2])
        df.columns = [
            'Регион',
            'Зарегистрировано ЗНО (без учтенных посмертно)',
            'Из них, диагноз подтвержден морфологически, %',
            'Из них, имели стадию заболевания, %, I',
            'Из них, имели стадию заболевания, %, II',
            'Из них, имели стадию заболевания, %, III',
            'Из них, имели стадию заболевания, %, IV',
            'Из них, имели стадию заболевания, %, не установлена',
            'Летальность на первом году с момента уст. диагноза, %'
        ]

        df['Индикатор'] = ind
        df['Локализация'] = loc
        df['Таблица'] = table
        df['Тип данных'] = 'СОП зарегистрировано продолжение'
        df['Год'] = year

        final_two_dop.append(df)
    else:
        print("Некорректный формат значения ячейки А1 в файле:", file)

Некорректный формат значения ячейки А1 в файле: 2021_Таблица_028_Состояние_онко_помощи_в_РФ.xlsx
Некорректный формат значения ячейки А1 в файле: 2021_Таблица_027_Состояние_онко_помощи_в_РФ.xlsx
Некорректный формат значения ячейки А1 в файле: 2021_Таблица_048_Состояние_онко_помощи_в_РФ.xlsx


In [272]:
final_two_dop_df = pd.concat(final_two_dop)


In [273]:
#  Удаление пробелов в начале и конце строк в столбце регион
final_two_dop_df.loc[:, 'Регион'] = final_two_dop_df['Регион'].str.strip()
# Удаление строки с названием "Россия"
final_two_dop_df = final_two_dop_df[final_two_dop_df['Регион'] != 'РОССИЯ']
# Извлекаем все строки, содержащие 'ФО'
bad_df = final_two_dop_df[final_two_dop_df['Регион'].str.upper().str.contains('ФО', regex=True)]
# Удаляем строки из final_two_df, которые есть в bad_df
final_two_dop_df = final_two_dop_df[~final_two_dop_df['Регион'].isin(bad_df['Регион'])]
# Создание нового столбца 'ФО' на основе существующего столбца 'Регион' с использованием лямбда-функции
final_two_dop_df['ФО'] = final_two_dop_df['Регион'].apply(lambda x: next((fd for fd, subs in fed_districts.items() if x in subs), None))



In [274]:
#  создадим 3 датафрейм, который будет состоять из 2 списков файлов: convert_three и convert_three_dop
# Определение пустого списка final_free
final_three = []

# Цикл для обработки каждого файла в convert_four
for el in tqdm(convert_three):
    # Открытие файла Excel
    workbook = openpyxl.load_workbook(el)

    # Выбор активного листа
    sheet = workbook.active

    # Извлечение значений из ячеек
    ind = sheet['A2'].value
    table = sheet['A1'].value
    loc = sheet['B4'].value
    year = sheet['B3'].value

    # Чтение данных из файла Excel в DataFrame
    df_three = pd.read_excel(el, header=[6])
    df_three.columns = ['Регион',
                  'Все население, абсолютное число',
                  'Все население,Показатель на 100 тыс. населения, "грубый"',
                  'Все население,Показатель на 100 тыс. населения, стандартизованный',
                  'Все население,Показатель на 100 тыс. населения, ошибка',
                  'Мужчины, абсолютное число',
                  'Мужчины, Показатель на 100 тыс. населения, "грубый"',
                  'Мужчины, Показатель на 100 тыс. населения, стандартизованный',
                  'Мужчины, Показатель на 100 тыс. населения, ошибка',
                  'Женщины, абсолютное число',
                  'Женщины, Показатель на 100 тыс. населения, "грубый"',
                  'Женщины, Показатель на 100 тыс. населения, стандартизованный',
                  'Женщины, Показатель на 100 тыс. населения, ошибка',]
    df_three['Индикатор'] = ind
    df_three['Локализация'] = loc
    df_three['Таблица'] = table
    df_three['Тип данных'] = 'ЗНО заболеваемость'
    df_three['Год'] = year

    # Добавление DataFrame в список final_four
    final_three.append(df_three)


100%|██████████| 39/39 [01:09<00:00,  1.78s/it]


In [275]:
final_three_df = pd.concat(final_three)


In [276]:
#  Удаление пробелов в начале и конце строк в столбце регион
final_three_df.loc[:, 'Регион'] = final_three_df['Регион'].str.strip()
# Удаление строки с названием "Россия"
final_three_df = final_three_df[final_three_df['Регион'] != 'РОССИЯ']
# Извлекаем все строки, содержащие 'ФО'
bad_df = final_three_df[final_three_df['Регион'].str.upper().str.contains('ФО', regex=True)]
# Удаляем строки из final_three_df, которые есть в bad_df
final_three_df = final_three_df[~final_three_df['Регион'].isin(bad_df['Регион'])]
# Создание нового столбца 'ФО' на основе существующего столбца 'Регион' с использованием лямбда-функции
final_three_df['ФО'] = final_three_df['Регион'].apply(lambda x: next((fd for fd, subs in fed_districts.items() if x in subs), None))


In [277]:
# Мы получили широкий датафрейм, в котором нужно выделить отдельно пол и разделить на 3 отдельных датафрейма
# Создание первого DataFrame
df_1 = final_three_df.loc[:, ['Регион', 'Все население, абсолютное число', 'Все население,Показатель на 100 тыс. населения, "грубый"', 'Все население,Показатель на 100 тыс. населения, стандартизованный', 'Все население,Показатель на 100 тыс. населения, ошибка', 'Индикатор', 'Локализация', 'Таблица', 'Тип данных', 'Год', 'ФО']]

# Создание второго DataFrame
df_2 = final_three_df.loc[:, ['Регион', 'Мужчины, абсолютное число', 'Мужчины, Показатель на 100 тыс. населения, "грубый"', 'Мужчины, Показатель на 100 тыс. населения, стандартизованный', 'Мужчины, Показатель на 100 тыс. населения, ошибка', 'Индикатор', 'Локализация', 'Таблица', 'Тип данных', 'Год', 'ФО']]

# Создание третьего DataFrame
df_3 = final_three_df.loc[:, ['Регион', 'Женщины, абсолютное число', 'Женщины, Показатель на 100 тыс. населения, "грубый"', 'Женщины, Показатель на 100 тыс. населения, стандартизованный', 'Женщины, Показатель на 100 тыс. населения, ошибка', 'Индикатор', 'Локализация', 'Таблица', 'Тип данных', 'Год', 'ФО']]


In [278]:
#  Добавим в каждый новый датафрейм колонку "Пол"
df_1['Пол'] = 'оба пола'
df_2['Пол'] = 'муж'
df_3['Пол'] = 'жен'

In [279]:
# Теперь объединим все эти три датафрейма, сделав в каждом из них одинаковые названия колонок
# Изменение названий столбцов
df_1.columns = ['Регион', 'Абсолютное число', 'Показатель на 100 тыс. населения, "грубый"', 'Показатель на 100 тыс. населения, стандартизованный',\
              'Показатель на 100 тыс. населения, ошибка', 'Индикатор', 'Локализация', 'Таблица', 'Тип данных', 'Год', 'ФО', 'Пол']

df_2.columns = ['Регион', 'Абсолютное число', 'Показатель на 100 тыс. населения, "грубый"', 'Показатель на 100 тыс. населения, стандартизованный',\
              'Показатель на 100 тыс. населения, ошибка', 'Индикатор', 'Локализация', 'Таблица', 'Тип данных', 'Год', 'ФО', 'Пол']
df_3.columns = ['Регион', 'Абсолютное число', 'Показатель на 100 тыс. населения, "грубый"', 'Показатель на 100 тыс. населения, стандартизованный',\
              'Показатель на 100 тыс. населения, ошибка', 'Индикатор', 'Локализация', 'Таблица', 'Тип данных', 'Год', 'ФО', 'Пол']


In [280]:
import numpy as np
# Определение пустого списка final_tree_dop
final_three_dop = []

# Цикл для обработки каждого файла в convert_four
for el in tqdm(convert_three_dop):
    # Открытие файла Excel
    workbook = openpyxl.load_workbook(el)

    # Выбор активного листа
    sheet = workbook.active

    # Извлечение значений из ячеек
    ind = sheet['A2'].value
    table = sheet['A1'].value.strip('')
    if sheet['B4'].value is not None and sheet['B4'].value != '':
        loc = sheet['B4'].value
    else:
        loc = sheet['A4'].value.strip()

    year = sheet['B3'].value

    # Чтение данных из файла Excel в DataFrame
    df_three_dop = pd.read_excel(el, header=[6])
    df_three_dop.columns = ['Регион',
                  'Абсолютное число',
                  'Показатель на 100 тыс. населения, "грубый"',
                  'Показатель на 100 тыс. населения, стандартизованный',
                  'Показатель на 100 тыс. населения, ошибка']
    df_three_dop['Индикатор'] = ind
    df_three_dop['Локализация'] = loc
    df_three_dop['Таблица'] = table
    df_three_dop['Тип данных'] = 'ЗНО заболеваемость'
    df_three_dop['Год'] = year
    df_three_dop['Пол'] = np.where((df_three_dop['Локализация'] == 'Предстательная железа (С61)') | (df_three_dop['Локализация'] == 'Другие мужские половые органы (С60,62,63)'), 'муж', 'жен')
    # Добавление DataFrame в список final_four
    final_three_dop.append(df_three_dop)

100%|██████████| 9/9 [00:01<00:00,  8.06it/s]


In [281]:
final_three_dop_df = pd.concat(final_three_dop)



In [282]:
#  Удаление пробелов в начале и конце строк в столбце регион
final_three_dop_df.loc[:, 'Регион'] = final_three_dop_df['Регион'].str.strip()
# Удаление строки с названием "Россия"
final_three_dop_df = final_three_dop_df[final_three_dop_df['Регион'] != 'РОССИЯ']
# Извлекаем все строки, содержащие 'ФО'
bad_df = final_three_dop_df[final_three_dop_df['Регион'].str.upper().str.contains('ФО', regex=True)]
# Удаляем строки из final_three_dop_df, которые есть в bad_df
final_three_dop_df = final_three_dop_df[~final_three_dop_df['Регион'].isin(bad_df['Регион'])]
# Создание нового столбца 'ФО' на основе существующего столбца 'Регион' с использованием лямбда-функции
final_three_dop_df['ФО'] = final_three_dop_df['Регион'].apply(lambda x: next((fd for fd, subs in fed_districts.items() if x in subs), None))
final_three_dop_df.columns = ['Регион', 'Абсолютное число', 'Показатель на 100 тыс. населения, "грубый"', 'Показатель на 100 тыс. населения, стандартизованный',\
              'Показатель на 100 тыс. населения, ошибка', 'Индикатор', 'Локализация', 'Таблица', 'Тип данных', 'Год', 'Пол', 'ФО']


In [283]:
#  Поменяем местами столбцы Пол и ФО
final_three_dop_df = final_three_dop_df.reindex(columns=['Регион', 'Абсолютное число',
       'Показатель на 100 тыс. населения, "грубый"',
       'Показатель на 100 тыс. населения, стандартизованный',
       'Показатель на 100 тыс. населения, ошибка', 'Индикатор', 'Локализация',
       'Таблица', 'Тип данных', 'Год', 'ФО', 'Пол'])


In [284]:
# Объединение DataFrame по вертикали
combined_data = pd.concat([df_1, df_2, df_3, final_three_dop_df ], axis=0)
# Обновление индексов
combined_data = combined_data.reset_index(drop=True)


In [285]:
#  В некоторых строках в Локализации есть пропуски:
xx = combined_data[combined_data['Локализация'].isna()]  # с помощью метода isna() найдем пропуски и выведем на экран строки с проопусками
xx['Таблица'].unique()


array(['Таблица 20', 'Таблица 51', 'Таблица 12'], dtype=object)

In [286]:
#  В таблицах 51, 20 и 12 Локализация была не в ячейке В4, а в ячейке А1. Поменяет руками локализацию:
combined_data.loc[combined_data['Таблица'] == "Таблица 51", 'Локализация'] = "Лимфогранулематоз (С81)"
combined_data.loc[combined_data['Таблица'] == "Таблица 20", 'Локализация'] = "Пищевод (С15)"
combined_data.loc[combined_data['Таблица'] == "Таблица 12", 'Локализация'] = "Злокачественные новообразования - всего (С00-96)"

In [287]:
# Вручную заменим пропуски в столбце Год на 2021
combined_data['Год'] = combined_data['Год'].fillna('2021')


In [288]:
#  Создадим 4 датафрейм, который будет состоять из 2 списков файлов: convert_four и convert_four_dop.
# Определение пустого списка final_four
final_four = []

# Цикл для обработки каждого файла в convert_four
for el in tqdm(convert_four):
    # Открытие файла Excel
    workbook = openpyxl.load_workbook(el)

    # Выбор активного листа
    sheet = workbook.active

    # Извлечение значений из ячеек
    ind = sheet['A2'].value
    table = sheet['A1'].value
    loc = sheet['B4'].value
    year = sheet['B3'].value

    # Чтение данных из файла Excel в DataFrame
    df_four = pd.read_excel(el, header=[6])
    df_four.columns = ['Регион',
                  'Все население, абсолютное число',
                  'Все население,Показатель на 100 тыс. населения, "грубый"',
                  'Все население,Показатель на 100 тыс. населения, стандартизованный',
                  'Все население,Показатель на 100 тыс. населения, ошибка',
                  'Мужчины, абсолютное число',
                  'Мужчины, Показатель на 100 тыс. населения, "грубый"',
                  'Мужчины, Показатель на 100 тыс. населения, стандартизованный',
                  'Мужчины, Показатель на 100 тыс. населения, ошибка',
                  'Женщины, абсолютное число',
                  'Женщины, Показатель на 100 тыс. населения, "грубый"',
                  'Женщины, Показатель на 100 тыс. населения, стандартизованный',
                  'Женщины, Показатель на 100 тыс. населения, ошибка',]
    df_four['Индикатор'] = ind
    df_four['Локализация'] = loc
    df_four['Таблица'] = table
    df_four['Тип данных'] = 'ЗНО смертность'
    df_four['Год'] = year

    # Добавление DataFrame в список final_four
    final_four.append(df_four)


100%|██████████| 29/29 [00:51<00:00,  1.76s/it]


In [289]:
final_four_df = pd.concat(final_four)


In [290]:
#  Удаление пробелов в начале и конце строк в столбце регион
final_four_df.loc[:, 'Регион'] = final_four_df['Регион'].str.strip()
# Удаление строки с названием "Россия"
final_four_df = final_four_df[final_four_df['Регион'] != 'РОССИЯ']
# Извлекаем все строки, содержащие 'ФО'
bad_df = final_four_df[final_four_df['Регион'].str.upper().str.contains('ФО', regex=True)]
# Удаляем строки из final_two_df, которые есть в bad_df
final_four_df = final_four_df[~final_four_df['Регион'].isin(bad_df['Регион'])]
# Создание нового столбца 'ФО' на основе существующего столбца 'Регион' с использованием лямбда-функции
final_four_df['ФО'] = final_four_df['Регион'].apply(lambda x: next((fd for fd, subs in fed_districts.items() if x in subs), None))


In [291]:
# Создание первого DataFrame
df1 = final_four_df.loc[:, ['Регион', 'Все население, абсолютное число', 'Все население,Показатель на 100 тыс. населения, "грубый"',\
                            'Все население,Показатель на 100 тыс. населения, стандартизованный', 'Все население,Показатель на 100 тыс. населения, ошибка',\
                            'Индикатор', 'Локализация', 'Таблица', 'Тип данных', 'Год', 'ФО']]

# Создание второго DataFrame
df2 = final_four_df.loc[:, ['Регион', 'Мужчины, абсолютное число', 'Мужчины, Показатель на 100 тыс. населения, "грубый"', \
                            'Мужчины, Показатель на 100 тыс. населения, стандартизованный', 'Мужчины, Показатель на 100 тыс. населения, ошибка', \
                            'Индикатор', 'Локализация', 'Таблица', 'Тип данных', 'Год', 'ФО']]

# Создание третьего DataFrame
df3 = final_four_df.loc[:, ['Регион', 'Женщины, абсолютное число', 'Женщины, Показатель на 100 тыс. населения, "грубый"', \
                            'Женщины, Показатель на 100 тыс. населения, стандартизованный', 'Женщины, Показатель на 100 тыс. населения, ошибка',\
                            'Индикатор', 'Локализация', 'Таблица', 'Тип данных', 'Год', 'ФО']]


In [292]:
df1['Пол'] = 'оба пола'
df2['Пол'] = 'муж'
df3['Пол'] = 'жен'


In [293]:
# Изменение названий столбцов
df1.columns = ['Регион', 'Абсолютное число', 'Показатель на 100 тыс. населения, "грубый"', 'Показатель на 100 тыс. населения, стандартизованный',\
              'Показатель на 100 тыс. населения, ошибка', 'Индикатор', 'Локализация', 'Таблица', 'Тип данных', 'Год', 'ФО', 'Пол']

df2.columns = ['Регион', 'Абсолютное число', 'Показатель на 100 тыс. населения, "грубый"', 'Показатель на 100 тыс. населения, стандартизованный',\
              'Показатель на 100 тыс. населения, ошибка', 'Индикатор', 'Локализация', 'Таблица', 'Тип данных', 'Год', 'ФО', 'Пол']
df3.columns = ['Регион', 'Абсолютное число', 'Показатель на 100 тыс. населения, "грубый"', 'Показатель на 100 тыс. населения, стандартизованный',\
              'Показатель на 100 тыс. населения, ошибка', 'Индикатор', 'Локализация', 'Таблица', 'Тип данных', 'Год', 'ФО', 'Пол']



In [294]:
# Определение пустого списка final_four_dop
final_four_dop = []

# Цикл для обработки каждого файла в convert_four
for el in tqdm(convert_four_dop):
    # Открытие файла Excel
    workbook = openpyxl.load_workbook(el)

    # Выбор активного листа
    sheet = workbook.active

    # Извлечение значений из ячеек
    ind = sheet['A2'].value
    table = sheet['A1'].value
    loc = sheet['B4'].value
    year = sheet['B3'].value

    # Чтение данных из файла Excel в DataFrame
    df_four_dop = pd.read_excel(el, header=[6])
    df_four_dop.columns = ['Регион',
                  'Абсолютное число',
                  'Показатель на 100 тыс. населения, "грубый"',
                  'Показатель на 100 тыс. населения, стандартизованный',
                  'Показатель на 100 тыс. населения, ошибка']
    df_four_dop['Индикатор'] = ind
    df_four_dop['Локализация'] = loc
    df_four_dop['Таблица'] = table
    df_four_dop['Тип данных'] = 'ЗНО смертность'
    df_four_dop['Год'] = year
    df_four_dop['Пол'] = np.where((df_four_dop['Локализация'] == 'Предстательная железа (С61)') | (df_four_dop['Локализация'] == 'Другие мужские половые органы (С60,62,63)'), 'муж', 'жен')
    # Добавление DataFrame в список final_four
    final_four_dop.append(df_four_dop)

100%|██████████| 6/6 [00:06<00:00,  1.14s/it]


In [295]:
final_four_dop_df = pd.concat(final_four_dop)


In [296]:
#  Удаление пробелов в начале и конце строк в столбце регион
final_four_dop_df.loc[:, 'Регион'] = final_four_dop_df['Регион'].str.strip()
# Удаление строки с названием "Россия"
final_four_dop_df = final_four_dop_df[final_four_dop_df['Регион'] != 'РОССИЯ']
# Извлекаем все строки, содержащие 'ФО'
bad_df = final_four_dop_df[final_four_dop_df['Регион'].str.upper().str.contains('ФО', regex=True)]
# Удаляем строки из final_four_dop_df, которые есть в bad_df
final_four_dop_df = final_four_dop_df[~final_four_dop_df['Регион'].isin(bad_df['Регион'])]
# Создание нового столбца 'ФО' на основе существующего столбца 'Регион' с использованием лямбда-функции
final_four_dop_df['ФО'] = final_four_dop_df['Регион'].apply(lambda x: next((fd for fd, subs in fed_districts.items() if x in subs), None))
final_four_dop_df.columns = ['Регион', 'Абсолютное число', 'Показатель на 100 тыс. населения, "грубый"', 'Показатель на 100 тыс. населения, стандартизованный',\
              'Показатель на 100 тыс. населения, ошибка', 'Индикатор', 'Локализация', 'Таблица', 'Тип данных', 'Год', 'Пол', 'ФО']


In [297]:
# Объединение DataFrame по вертикали
combined_df = pd.concat([df1, df2, df3, final_four_dop_df ], axis=0)
# Обновление индексов
combined_df = combined_df.reset_index(drop=True)


In [298]:
#  В таблице 71 ЗНО локализация болезни была не в ячейке В4, а в ячейке А4.
#  Соответственно, данные не подтянулась в датафрейм. Вместо пропусков поставим "Пищевод".
#  Можно рекомендовать защитить ячейки в таблицах, куда вносят данные, чтобы для локализации была только ячейка B4.
combined_df['Локализация'] = combined_df['Локализация'].fillna('Пищевод')


In [299]:
#  Теперь соединим combined_data в котором содержаться сведения о ЗНО заболеваемости и combined_df со сведениями о ЗНО смертности в датафрейм ZNO:
ZNO = pd.concat([combined_data, combined_df], axis=0)
# Обновление индексов
ZNO = ZNO.reset_index(drop=True)


In [300]:
#  Сохраним датафреймы в excel файлы
final_one_df.to_excel('Состояние онко помощи (СОП) в РФ (впервые выявленно).xlsx', index=False)
final_two_df.to_excel('Состояние онко помощи (СОП) в РФ (состоящие на учете).xlsx', index=False)
final_two_dop_df.to_excel('Состояние онко помощи (СОП) в РФ (состоящие на учете_продолжение).xlsx', index=False)
ZNO.to_excel('Злокачественные новообразования (ЗНО) в РФ (заболеваемость и смертность).xlsx', index=False)


In [301]:
from google.colab import files

files.download('Состояние онко помощи (СОП) в РФ (впервые выявленно).xlsx')
files.download('Состояние онко помощи (СОП) в РФ (состоящие на учете).xlsx')
files.download('Состояние онко помощи (СОП) в РФ (состоящие на учете_продолжение).xlsx')
files.download('Злокачественные новообразования (ЗНО) в РФ (заболеваемость и смертность).xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>